SETUP THE ENVIRONMENT

In [1]:
!pip install -U pip
!pip install appdirs
!pip install datasets
!pip install fire
!pip install -U torch
!pip install sentencepiece
!pip install tensorboardX
!pip install gradio
!pip install -U transformers tokenizers
!pip install scipy
!pip install seaborn
!pip install evaluate
!pip install -U pytorch-lightning
!pip install bitsandbytes
!pip install -q huggingface_hub
!pip install -q -U trl accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/66/f8/38298237d18d4b6a8ee5dfe390e97bed5adb8e01ec6f9680c0ddf3066728/datasets-2.14.4-py3-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/13/c3/e942893f4864a424514c81640f114980cfd5aff7e7414d1e0255f4571111/xxhash-3.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/35/a8/36d8d7b3e46b377800d8dec4789

In [2]:
!wandb login
!cat /root/.netrc

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: ERROR API key must be 40 characters long, yours was 4095
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: ERROR API key must be 40 characters long, yours was 4095
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.

IMPORT AND INITIALISE MODEL AND TOKENIZER

In [3]:
from huggingface_hub import notebook_login

notebook_login()


In [4]:
%env WANDB_PROJECT=llama-2-7b-hf-zero-shot-no-prompt
from datasets import load_from_disk
from torch.cuda import amp
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from torch.cuda.amp import autocast, GradScaler
from transformers import AdamW, get_scheduler
from torch import nn
import gc
from transformers import (
    AdamW,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    set_seed,
)
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
from datasets import concatenate_datasets, load_dataset
from transformers import AdamW,get_scheduler
import torch
from tqdm.auto import tqdm
import os
import transformers
import textwrap
from transformers import Trainer
from datasets import load_dataset
from transformers import DataCollatorWithPadding
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    TaskType
)
from torch.utils.data import DataLoader

import fire
import torch
from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams
import json
import datasets
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, \
    DataCollatorForTokenClassification,LlamaForSequenceClassification


env: WANDB_PROJECT=llama-2-7b-hf-zero-shot-no-prompt


In [5]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
dataset_name = "niting3c/malicious-packet-analysis"
new_model = "niting3c/llama-2-7b-hf-zero-shot-prompt"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 100
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 3e-4
weight_decay = 0.01
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 40
logging_steps = 40
max_seq_length = 4096
packing = False
device_map = {"": 0}

In [7]:
import datasets
candidate_labels = ["attack", "normal"]

model_features = datasets.Features(
        {'text': datasets.Value('string'), 'label': datasets.ClassLabel(num_classes=2,names=candidate_labels)})



In [ ]:
train_dataset1 = load_dataset("niting3c/malicious-packet-analysis", data_dir='network-packet-flow-header-payload',split="train",features=model_features).select(range(3000))
train_dataset2 = load_dataset("niting3c/malicious-packet-analysis", data_dir='normal_netresc',split="train",features=model_features).select(range(1000))
val_dataset = load_dataset("niting3c/malicious-packet-analysis", data_dir='network-packet-flow-header-payload',split="test",features=model_features).select(range(1500))
train_dataset= concatenate_datasets([train_dataset1,train_dataset2])

In [ ]:
val_dataset

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['text'])):
        text = f"### prompt: {example['text'][i]}\n ### Response: {example['label'][i]}"
        output_texts.append(text)
    return output_texts

In [ ]:
import torch
torch.cuda.empty_cache()

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
        bf16 = True


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.config.architectures = ["LlamaForSequenceClassification"]
model.config.zero_shot_classification = True
label2id = {label: i for i, label in enumerate(candidate_labels)}
id2label = {i: label for i, label in enumerate(candidate_labels)}

# Set the model's label mapping
model.config.label2id = label2id
model.config.id2label = id2label

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


In [ ]:

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    hub_model_id="niting3c/llama-2-7b-hf-zero-shot-prompt",
    push_to_hub=True,
    load_best_model_at_end=True,
    eval_steps=20  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    formatting_func=formatting_prompts_func,
    packing=packing,

)
trainer.train()

In [ ]:
## F1 Score, Precision , Recall , Accuracy
tokenizer.push_to_hub("niting3c/llama-2-7b-hf-zero-shot-prompt", use_auth_token=True)